<a href="https://colab.research.google.com/github/georgiam11/predict-gender/blob/main/FINAL_GENDER_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This green text is a comment. By using '#' before typing, nothing I use will affect the code.
#Run, but do not edit
from google.colab import drive
drive.mount('/content/drive')
#sign into google account if prompted and allow all

Mounted at /content/drive


In [ ]:
#Run, but do not edit
#import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import os
#import seaborn as sns

In [ ]:
# if you wish to change 'wage_info' you must do so for the rest of the code, do CMD/CNTRL+H and replace all wage_info to the desired name.
#wage_info = pd.read_csv('/conent/drve/My Drive/FOLDER NAME/DATASET NAME.csv') --> like stated in the slideshow, this is the map you're giving your code to the dataset
wage_info = pd.read_csv('/content/drive/My Drive/Vanderbilt_Info/July_Union_Roster.csv')

In [ ]:
!pip install transformers
#this and the next few blocks might take a while depending on your ram and gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

classifier = pipeline("fill-mask", model="distilroberta-base", device=0)
#If this does not run correctly, go to the top right and click on the dropdown next to the two rectangles named RAM and DISK with green lines through them.
#Click "change runtime type" and change it to T4 GPU and click save.

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def BERTname(name):
  results = classifier(name+" is <mask> name.", targets=["Ġher", "Ġhis"])
  results.sort(key=lambda x: x['token_str'])
  return (results[0]['score'], results[1]['score'])
#run, but do not edit

In [ ]:
def BERT_classify(her):
  if her > 0.5: return "F"
  return "M"
#also run but dont edit

In [ ]:
def fix_ratio(ratio, gender):
  if gender == 'F': return ratio
  return 1 - ratio
#run dont edit

In [ ]:
#wage_info['BERT'] = wage_info['COLUMN NAME THAT CONTAINS THE NAMES'].apply(BERTname)
wage_info['BERT'] = wage_info['Name'].apply(BERTname)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:

wage_info['her'] = [x[0] for x in wage_info['BERT']]
wage_info['his'] = [x[1] for x in wage_info['BERT']]
del wage_info['BERT']
wage_info['BERTtotal'] = wage_info['his'] + wage_info['her']
wage_info['his'] /= wage_info['BERTtotal']
wage_info['her'] /= wage_info['BERTtotal']
del wage_info['BERTtotal']
wage_info["BERT"] = wage_info["her"].apply(BERT_classify)

In [ ]:
wage_info = wage_info.merge(wage_info, on="Name", how="left")
#wage_info.to_csv("NAME OF DATASET THAT IS ON GOOGLE DRIVE, SAME NAME USED IN 3RD BLOCK", index=False)
wage_info.to_csv("July_Union_Roster.csv", index=False) #adds row with the his/her ratio and predicted gender
wage_info.head()

,Name,Assignment Status_x_x,Business Unit Name_x_x,Work Unit_x_x,Person Number_x_x,Job_x_x,Work Relationship Start Date_x_x,Full/Part Time_x_x,Hourly Rate_x_x,Person Gender_x_x,...,Region 1_y_y,City_y_y,Region 2_y_y,Postal Code_y_y,Person Ethnicity_y_y,Phone_y_y,Legal Employer Seniority Date_y_y,her_y_y,his_y_y,BERT_y_y
0,"Daniels, Dwayne",Active - Payroll Eligible,Business Services,36520 - Catering Services,86768,Supply Delivery Technician,3/2/2009,Full time,21.4,Male,...,Davidson,Nashville,TN,37208,Black or African American,1-615-445-5568,3/2/2009,0.118711,0.881289,M
1,"Tisdale, Gregory",Active - Payroll Eligible,Business Services,36520 - Catering Services,150996,Food Service Worker,11/13/2017,Full time,17.4,Male,...,Sumner,Goodlettsville,TN,37072,Black or African American,1-615-596-8848,11/13/2017,0.244879,0.755121,M
2,"Burum, Breland",Active - Payroll Eligible,Business Services,36520 - Catering Services,3044765,Cook,2/10/2023,Full time,19.0,Male,...,Davidson,Nashville,TN,37221,Black or African American,1-865-2468920,2/10/2023,0.276390,0.723610,M
3,"Tuberville, Jerridene",Active - Payroll Eligible,Business Services,36520 - Catering Services,3044769,Cook,2/13/2023,Full time,19.0,Female,...,Cheatham,Ashland City,TN,37015,White,1-615-5413044,2/13/2023,0.549222,0.450778,F
4,"Burton, Dwayne",Active - Payroll Eligible,Business Services,36520 - Catering Services,3047066,Senior Cook,4/4/2023,Full time,21.5,Male,...,Davidson,Nashville,TN,37216,Black or African American,1-561-3559724,4/4/2023,0.058945,0.941055,M
